In [1]:
# * IMPORTS
import os
import threading
import time
from datetime import datetime, time
from ipywidgets import widgets
from functools import partial
from config import StyleConfig
from IPython.display import display, clear_output, HTML
# from visuals.map_manager import createBaseMap
from visuals.gui_manager import reset_cbox, get_barrio_names


import random
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
from rdflib.plugins.sparql import prepareQuery

dict_namespaces = {
    "rr": Namespace("http://www.w3.org/ns/r2rml#"),
    "rml": Namespace("http://semweb.mmlab.be/ns/ql#"),
    "ql": Namespace("http://vocab.org/transit/terms/"),
    "transit": Namespace("http://www.w3.org/2001/XMLSchema#"),
    "wgs84_pos": Namespace("http://www.w3.org/2003/01/geo/wgs84_pos#"),
    "vocab": Namespace("http://example.org#"),
    "dog-loc": Namespace("https://w3id.org/DogFriendlyMadrid/info/ontology/location#"),
    "dog-det": Namespace("https://w3id.org/DogFriendlyMadrid/info/ontology/details#"),
    "dog-ser": Namespace("https://w3id.org/DogFriendlyMadrid/info/ontology/services#"),
    "schema-org": Namespace("http://schema.org/"),
    "dbo": Namespace("https://dbpedia.org/ontology/"),
    "rdfs": Namespace("http://www.w3.org/2000/01/rdf-schema#"),
    "rdf": Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#"),
    "owl": Namespace("http://www.w3.org/2002/07/owl#")
}

graph_Parks = Graph()
graph_CareCenter = Graph()
graph_DogGarbage = Graph()
graph_DogZone = Graph()
graph_Fountain = Graph()

for shortcut, new_namespace in dict_namespaces.items():
    graph_Parks.namespace_manager.bind(shortcut, new_namespace, override=False)
for shortcut, new_namespace in dict_namespaces.items():
    graph_CareCenter.namespace_manager.bind(shortcut, new_namespace, override=False)
for shortcut, new_namespace in dict_namespaces.items():
    graph_DogGarbage.namespace_manager.bind(shortcut, new_namespace, override=False)
for shortcut, new_namespace in dict_namespaces.items():
    graph_DogZone.namespace_manager.bind(shortcut, new_namespace, override=False)
for shortcut, new_namespace in dict_namespaces.items():
    graph_Fountain.namespace_manager.bind(shortcut, new_namespace, override=False)

graph_Parks.parse("./Parks-with-links.nt", format="nt")
graph_CareCenter.parse("./CareCenters-with-links.nt", format="nt")
graph_DogGarbage.parse("./DogGarbageBins-with-links.nt", format="nt")
graph_DogZone.parse("./DogZones-with-links.nt", format="nt")
graph_Fountain.parse("./FuentesMascotas-with-links_data.nt", format="nt")

<Graph identifier=N118123d2bdf748bfa87c5b34610d8c9a (<class 'rdflib.graph.Graph'>)>

In [2]:
# we check if we run in a notebook or a voila app
# we will not display the map by default if in a voila app
running_in_voila = os.environ.get('SERVER_SOFTWARE','jupyter').startswith('voila')


In [4]:
# * WIDGETS

# header image

with open("./images/hero_cropped.jpg", 'rb') as file:
        image = file.read()

header_img = widgets.Image(
    value=image,
    format='jpg',
)
header_img.layout.height = '200px'

hbox_image = widgets.HBox([widgets.Label(), header_img, widgets.Label()], 
                          layout=widgets.Layout(justify_content='center'))

btn_search = widgets.Button(
    description = "Busca",
    disabled = True,
    tooltip = "Busca DogCity",
    icon = "search"
)
# map button (only shown in Voila app)
btn_map = widgets.Button(
    description = "Mostrar mapa",
    disabled = False,
    tooltip = "Mostrar Mapa",
    icon = "map"
)
# progress_bar
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Generating map:',
    style={'bar_color': 'blue'},
    orientation='horizontal'
)



# output for the map, so we can change it
out_map = widgets.Output()

barrio_data = get_barrio_names(graph_Parks, dict_namespaces)
barrio_names = barrio_data["barrio_names"]

barrios = widgets.Combobox(
    placeholder="Elige el barrio",
    options= barrio_names,
    description="Elige tu barrio",
    ensure_option=True,
    disabled=False
    )
def barrios_observable(change):
    print(change)
    print(change["new"])
    index = barrio_names.index(change["new"])
    print(index)

# we set what to do when the checkbox changes
barrios.observe(barrios_observable, names='value')

journey_tab = widgets.VBox([widgets.Label(), barrios])
# the tabs widget that starts with only one tab



tabs = widgets.Tab(children=[journey_tab])
tabs.set_title(0, 'Buscador')
tabs.layout.justify_content = 'center'
# placement of the tabs widget in the middle of the page
hbox_tabs = widgets.HBox([widgets.Label(), tabs, widgets.Label()], layout=widgets.Layout(justify_content='center'))



IndentationError: unindent does not match any outer indentation level (<tokenize>, line 57)

In [ ]:
# * MAIN DISPLAY

display(HTML(StyleConfig.MATERIAL_CSS.value))
display(hbox_image)

display(hbox_tabs)
display(out_map)




def on_button_click_map(btn):
    with out_map:
        clear_output(wait=True)
        display(initial_map)


btn_map.on_click(on_button_click_map)


Output()